In [39]:
import requests
import csv
import pandas as pd
import os
import geopandas as gpd

# Base URL for the ACS data
base_url = "https://api.census.gov/data/{year}/acs/acs5"
subject_url = "https://api.census.gov/data/{year}/acs/acs5/subject"
profile_url = "https://api.census.gov/data/{year}/acs/acs5/profile"

# Places to change
# (1) variables (2) relative path

# Variables to fetch

# Economics #

# Median Income
#variables = {'main_median_income': "NAME,S1903_C03_001E", 'sub_median_white': "NAME,S1903_C03_002E",
             #'sub_median_black': "NAME,S1903_C03_003E", 'sub_median_ind_ala': "NAME,S1903_C03_004E",
             #'sub_median_asia': "NAME,S1903_C03_005E", 'sub_median_hawai': "NAME,S1903_C03_006E",
             #'sub_median_other': "NAME,S1903_C03_007E"  }

# Mean Income
#variables = {'main_mean_income': "NAME,S1902_C03_019E", 'sub_mean_white': "NAME,S1902_C03_020E",
             #'sub_mean_black': "NAME,S1902_C03_021E", 'sub_mean_ind_ala': "NAME,S1902_C03_022E",
             #'sub_mean_asia': "NAME,S1902_C03_023E", 'sub_mean_hawai': "NAME,S1902_C03_024E",
             #'sub_mean_other': "NAME,S1902_C03_025E"}

# Housing #

# HouseRent
# variables = {'main_agg_rent': "NAME,B25060_001E", 'sub_median_rent': "NAME,B25058_001E", 
             #'sub_lower_rent': "NAME,B25057_001E", 'sub_upper_rent': "NAME,B25059_001E"}
# HouseholdType
#variables = {'main_household_total': "NAME,B11001_001E", 'sub_household_family': "NAME,B11001_002E", 
             #'sub_household_nonfamily': "NAME,B11001_007E"}


# Education
# median earning
#variables = {'main_median_earning': "NAME,S1501_C01_059E", 'sub_less_high': "NAME,S1501_C01_060E",
             #'sub_high': "NAME,S1501_C01_061E", 'sub_college': "NAME,S1501_C01_062E",
             #'sub_bachelor': "NAME,S1501_C01_063E", 'sub_grad': "NAME,S1501_C01_064E"}

# enrollment
#variables = {'main_enroll': "NAME,S1401_C01_001E", 'sub_nursery': "NAME,S1401_C01_002E",
             #'sub_kind_12': "NAME,S1401_C01_003E", 'sub_college': "NAME,S1401_C01_008E",
             #'sub_grad': "NAME,S1401_C01_009E"}

# Health #
# Disability
#variables = {'main_disability': "NAME,S1810_C02_001E", 'sub_hearing': "NAME,S1810_C02_019E",
             #'sub_vision': "NAME,S1810_C02_029E", 'sub_cognitive': "NAME,S1810_C02_039E",
             #'sub_ambulatory': "NAME,S1810_C02_047E", 'sub_self_care': "NAME,S1810_C02_055E",
             #'sub_ind_living': "NAME,S1810_C02_063E"}

# Insurance
#variables = {'main_population': "NAME,S2701_C01_001E", 'sub_insured': "NAME,S2701_C02_001E",
             #'sub_uninsured': "NAME,S2701_C04_001E"}

# Population #
# Races

#variables = {'main_population': "NAME,DP05_0033E", 'sub_pop_white': "NAME,DP05_0037E",
             #'sub_pop_black': "NAME,DP05_0038E", 'sub_pop_ind_ala': "NAME,DP05_0039E",
             #'sub_pop_asia': "NAME,DP05_0044E", 'sub_pop_hawai': "NAME,DP05_0052E",
             #'sub_pop_other': "NAME,DP05_0057E", 'sub_pop_two': "NAME,DP05_0058E"}

variables = {'main_median_age': "NAME,B01002_001E", 'sub_male_age': "NAME,B01002_002E",
             'sub_female_age': "NAME,B01002_003E"}

# Location filters
location = "for=tract:*&in=state:17+county:031"

In [40]:
# Helper Function-1

def extract_info_from_filename(filename, ind_type):
    # Assuming filename is something like 'main_2017.csv'
    parts = filename.split('_')
    if len(parts) > 1:
        year_part = parts[-1]  # This would be '2017.csv'
        year = year_part.split('.')[0]  # This splits '2017.csv' into '2017' and 'csv' and takes the first part
        if ind_type == 'main':
            indicator = filename[5:-9]
        else:
            indicator = filename[4:-9]
        if year.isdigit():  # Check if 'year' is all digits
            return (indicator, int(year))
    return None, None

all_dataframes = []

In [41]:
for name, variable in variables.items():    

    ind_type = name.split('_')[0]

    for year in range(2017, 2023):
        # Construct the URL for the current year
        if variable[5:][0] == 'B':    
            url = f"{base_url.format(year=year)}?get={variable}&{location}"
        elif variable[5:][0] == 'S':
            url = f"{subject_url.format(year=year)}?get={variable}&{location}"
        else:
            url = f"{profile_url.format(year=year)}?get={variable}&{location}"
            
        # Make the request
        response = requests.get(url)

        data = response.json()

        current_dir = os.getcwd()

        # relative path specifically for Contract Rent indicator
        relative_path = os.path.join('data_downloaded', 'Population', 'MedianAge')

        # Combine the current directory with the relative path
        full_base_path = os.path.join(current_dir, relative_path)

        # Specify the path to save the CSV file, one for each year
        file_path = os.path.join(full_base_path, f'{ind_type}/{name}_{year}.csv')
        
        # Open a CSV file for writing for each year
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)

            # Write the data into the CSV file
            writer.writerows(data)

        print(f"Data for {year} for {name} has been written to {file_path}")

Data for 2017 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/MedianAge/main/main_median_age_2017.csv
Data for 2018 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/MedianAge/main/main_median_age_2018.csv
Data for 2019 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/MedianAge/main/main_median_age_2019.csv
Data for 2020 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/MedianAge/main/main_median_age_2020.csv
Data for 2021 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/MedianAge/main/main_median_age_2021.csv
Data for 2022 for main_median_age has been written to /home/yujie0706/DataForGood-chicago/dfg_chi/backend/data_downloaded/Population/Me

In [42]:
folder_path = "data_downloaded/Population/MedianAge/"

for ind_type in ['main', 'sub']:
    
    short_dir = os.path.join(folder_path, f'{ind_type}')

    if ind_type == 'main':    

        for filename in os.listdir(short_dir):

            if filename.endswith('.csv'):

                filepath = os.path.join(short_dir, filename)
            
                indicator_name, year = extract_info_from_filename(filename, ind_type)

                if indicator_name and year:
                    df = pd.read_csv(filepath)
                    # Create a new DataFrame with the required columns
                    new_df = pd.DataFrame({
                        'indicator_id': [int(2)] * len(df),
                        'census_tract_id': df['tract'].astype(int),
                        'indicator_name': indicator_name,
                        'year': int(year),
                        'value': df.iloc[:, 1].fillna(0).astype(int)
                    })
                    
                    all_dataframes.append(new_df)

        # Concatenate all DataFrames
        final_dataframe = pd.concat(all_dataframes)

        # Save to a new CSV file
        final_dataframe.to_csv(os.path.join(short_dir, 'Main_Agg.csv'), index = False)
        print('aggregate table saved')

    else:

        all_sub_dataframes = []  # List to store each file's DataFrame

        for filename in os.listdir(short_dir):
            if filename.startswith('sub') and filename.endswith('.csv'):
                filepath = os.path.join(short_dir, filename)
                sub_indicator_name, year = extract_info_from_filename(filename, ind_type)
        
                if sub_indicator_name and year:
                    df = pd.read_csv(filepath)
                    df['tract'] = pd.to_numeric(df['tract'], errors='coerce').fillna(0).astype(int)
                    df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
            
            # Create a new DataFrame with the required columns
                    new_sub_df = pd.DataFrame({
                        'indicator_id': [int(2)] * len(df),
                        'census_tract_id': df['tract'].astype(int),
                        'sub_group_indicator_name': sub_indicator_name,
                        'year': int(year),
                        'value': df.iloc[:, 1].fillna(0).astype(int)
            })
                    
                    # Append this new DataFrame to the list
                    all_sub_dataframes.append(new_sub_df)

        # Concatenate all DataFrames for 'sub' into one
        final_sub_dataframe = pd.concat(all_sub_dataframes, ignore_index=True)

        # Save to a new CSV file
        final_sub_dataframe.to_csv(os.path.join(short_dir, 'Sub_Agg.csv'), index=False)

        print('sub_aggregate table saved')

aggregate table saved
sub_aggregate table saved


/tmp/user/23071/ipykernel_3258110/3794545168.py:49: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3794545168.py:49: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3794545168.py:49: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. 

In [43]:
# save all agg files to a separate folder
data_downloaded_path = "data_downloaded"
indicator_id_map = {"Economics": 1, "Education": 2, "Health": 3, "Housing": 4, "Population": 5}
output_dir = os.path.join(data_downloaded_path, 'Agg_data')
os.makedirs(output_dir, exist_ok=True)

# Recursively search for CSV files in all subfolders
for root, dirs, files in os.walk(data_downloaded_path):
    agg_main_dataframes = []  # List to store dataframes for Agg_Main.csv
    agg_sub_dataframes = []   # List to store dataframes for Agg_Sub.csv

    # Check if the current directory is a largest directory under 'data_downloaded'
    if os.path.dirname(root) == data_downloaded_path:
        # Get the corresponding indicator_id for each directory
        indicator_id = indicator_id_map.get(os.path.basename(root))
        if indicator_id is None:
            continue

    if root.count('/') == 2:
        sub_ind = root.strip('/').split('/')[-1]

    for filename in files:
        if filename.endswith('.csv'):
            filepath = os.path.join(root, filename)
            ind_type = os.path.basename(os.path.dirname(filepath))

            indicator_name, year = extract_info_from_filename(filename, ind_type)

            if indicator_name and year:
                df = pd.read_csv(filepath)

                # Do we want to fill NAs with 0?
                df['tract'] = pd.to_numeric(df['tract'], errors='coerce').fillna(0).astype(int)
                df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)

                # Create dataframes for agg CSVs
                if ind_type == 'main':
                    new_main_df = pd.DataFrame({
                        'indicator_id': [indicator_id] * len(df),
                        'census_tract_id': df['tract'].astype(int),
                        'indicator_name': indicator_name,
                        'year': int(year),
                        'value': df.iloc[:, 1].fillna(0).astype(int)
                    })
                    agg_main_dataframes.append(new_main_df)
                elif ind_type == 'sub':
                    new_sub_df = pd.DataFrame({
                        'indicator_id': [indicator_id] * len(df),
                        'census_tract_id': df['tract'].astype(int),
                        'sub_group_indicator_name': indicator_name,
                        'year': int(year),
                        'value': df.iloc[:, 1].fillna(0).astype(int)
                    })
                    agg_sub_dataframes.append(new_sub_df)

    # Concatenate all DataFrames for Agg_Main.csv and Agg_Sub.csv
    if agg_main_dataframes:
        final_main_dataframe = pd.concat(agg_main_dataframes, ignore_index=True)
        output_filename = '_'.join([sub_ind, 'Main.csv'])
        final_main_dataframe.to_csv(os.path.join(output_dir, output_filename), index=False)
        print(f'Aggregated Main CSV file saved: {output_filename}')

    if agg_sub_dataframes:
        final_sub_dataframe = pd.concat(agg_sub_dataframes, ignore_index=True)
        output_filename = '_'.join([sub_ind, 'Sub.csv'])
        final_sub_dataframe.to_csv(os.path.join(output_dir, output_filename), index=False)
        print(f'Aggregated Sub CSV file saved: {output_filename}')

Aggregated Sub CSV file saved: Enrollment_Sub.csv
Aggregated Main CSV file saved: Enrollment_Main.csv
Aggregated Main CSV file saved: MedianEarning_Main.csv
Aggregated Sub CSV file saved: MedianEarning_Sub.csv
Aggregated Sub CSV file saved: HouseholdType_Sub.csv
Aggregated Main CSV file saved: HouseholdType_Main.csv


/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. 

Aggregated Sub CSV file saved: ContractRent_Sub.csv
Aggregated Main CSV file saved: ContractRent_Main.csv
Aggregated Sub CSV file saved: MeanIncome_Sub.csv
Aggregated Main CSV file saved: MeanIncome_Main.csv
Aggregated Main CSV file saved: MedianIncome_Main.csv
Aggregated Sub CSV file saved: MedianIncome_Sub.csv
Aggregated Main CSV file saved: MedianAge_Main.csv
Aggregated Sub CSV file saved: MedianAge_Sub.csv


/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 1] = pd.to_numeric(df.iloc[:, 1], errors='coerce').fillna(0).astype(int)
/tmp/user/23071/ipykernel_3258110/3546755849.py:34: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. 

Aggregated Sub CSV file saved: Races_Sub.csv
Aggregated Main CSV file saved: Races_Main.csv
Aggregated Main CSV file saved: Insurance_Main.csv
Aggregated Sub CSV file saved: Insurance_Sub.csv
Aggregated Main CSV file saved: Disability_Main.csv
Aggregated Sub CSV file saved: Disability_Sub.csv


In [44]:
# This part is for checking the subgroups of each indicator # 
# Test Only #

import os
import glob

# Replace 'path/to/your/folder' with the actual path to your folder
folder_path = os.path.join(os.getcwd(), 'data_downloaded', 'Agg_data')

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Loop through the list of CSV files
for csv_file in csv_files:
        
    # Read the CSV file into a DataFrame
        file_name = os.path.basename(csv_file)


        if file_name not in ['CensusTracts.csv', 'TractZipCodes.csv']:

            df = pd.read_csv(csv_file)
            main_part = file_name.split('_')[1].replace('.csv', '')

            if main_part == 'Main':
                column_name = 'indicator_name'  
                unique_categories = df[column_name].unique()
            else:
                column_name = 'sub_group_indicator_name'  
                unique_categories = df[column_name].unique()
            
            print(f"'{file_name}': {unique_categories}")




'Enrollment_Main.csv': ['enroll']
'ContractRent_Main.csv': ['agg_rent']
'Enrollment_Sub.csv': ['kind_12' 'grad' 'nursery' 'college']
'MedianEarning_Sub.csv': ['less_high' 'high' 'bachelor' 'college' 'grad']
'Disability_Sub.csv': ['cognitive' 'hearing' 'ambulatory' 'vision' 'ind_living' 'self_care']
'Insurance_Sub.csv': ['uninsured' 'insured']
'MedianAge_Main.csv': ['median_age']
'MedianEarning_Main.csv': ['median_earning']
'MedianAge_Sub.csv': ['male_age' 'female_age']
'Disability_Main.csv': ['disability']
'Races_Sub.csv': ['pop_asia' 'pop_black' 'pop_white' 'pop_two' 'pop_hawai' 'pop_ind_ala'
 'pop_other']
'MedianIncome_Sub.csv': ['median_hawai' 'median_other' 'median_asia' 'median_ind_ala'
 'median_white' 'median_black']
'MeanIncome_Main.csv': ['mean_income']
'HouseholdType_Sub.csv': ['household_family' 'household_nonfamily']
'Races_Main.csv': ['population']
'HouseholdType_Main.csv': ['household_total']
'MedianIncome_Main.csv': ['median_income']
'MeanIncome_Sub.csv': ['mean_hawai' 'm

### Data quality check

Make sure different data sources are consistent on tracts and zipcodes

In [ ]:
tract_shapefile_path = os.path.join(os.getcwd(), os.path.pardir, 'censustracts/censustracts.shp')
tract_gdf = gpd.read_file(tract_shapefile_path)
tract_gdf['tractce10'] = tract_gdf['tractce10'].astype(int)

tract_file_path = os.path.join(os.getcwd(), 'data_downloaded/Agg_data/CensusTracts.csv')
census_tract_df = pd.read_csv(tract_file_path)

assert census_tract_df['tract_id'].unique() in tract_gdf['tractce10'].unique()

In [ ]:
zip_shapefile_path = os.path.join(os.getcwd(), os.path.pardir, 'zipcode/zipcode.shp')
zip_gdf = gpd.read_file(zip_shapefile_path)
zip_gdf['zip'] = zip_gdf['zip'].astype(int)

zip_file_path = os.path.join(os.getcwd(), 'data_downloaded/Agg_data/TractZipCodes.csv')
zipcode_df = pd.read_csv(zip_file_path)

assert zipcode_df['zip_code'].unique() in zip_gdf['zip'].unique()

Check missing values and remove tract id not in CensusTracts 

In [ ]:
folder_path = 'data_downloaded/Agg_data'
# census_tract_df = pd.read_csv("data_downloaded/Agg_data/CensusTracts.csv")
valid_tract_ids = set(census_tract_df['tract_id'])


# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        if 'value' in df:
            # Replace negative values with NaN (which will be saved as NULL in the CSV)
            df['value'] = df['value'].apply(lambda x: pd.NA if (x < 0 or pd.isna(x)) else int(x))
            # filter out tract 80400
            if 'census_tract_id' in df or 'zip_code' in df:
                df = df[df['census_tract_id'].isin(valid_tract_ids)]
    
            # Save the updated DataFrame back to the CSV file
            df.to_csv(file_path, index=False)

print("All CSV files in the 'Agg_data' folder have been updated.")

Double check for negative values

In [ ]:
all_negative_values = set()  # Use a set to store unique negative values


for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Read the 'value' column, skip rows with missing 'value' data
            df = pd.read_csv(file_path, usecols=['value'])
            
            # Filter out negative values and drop NA/None
            negative_values = df['value'].dropna()[df['value'] < 0]
            
            # Update the set of all unique negative values
            all_negative_values.update(negative_values.unique())
        
        except pd.errors.EmptyDataError:
            print(f"File '{filename}' is empty or missing critical data.")
        except FileNotFoundError:
            print(f"File '{filename}' not found.")
        except Exception as e:
            print(f"An error occurred while processing file '{filename}': {str(e)}")

# Print all unique negative values found in all files
print("All unique negative values across all CSV files:")
for value in sorted(all_negative_values):
    print(value)